## Extracting Data from Pandas dataframe to MySql Table

#### Here's a typical project flow:
####          1.Data Cleaning in Python (Using Pandas)
####          2.Store Cleaned Data in SQL database
####          3.Perform SQL operations

In [51]:
# Importing Dependencies

import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

## Data

In [2]:
df = pd.read_csv('./Employee Data.csv', encoding='latin-1')

#if you got trouble while reading csv file mention encoding as 'latin-1'

In [3]:
df.head()

,Employee ID,Full Name,Job Title,Department,Business Unit,Gender,Ethnicity,Age,Hire Date,Annual Salary,Bonus %,Country,City,Exit Date
0,E02002,Kai Le,Controls Engineer,Engineering,Manufacturing,Male,Asian,47.0,2/5/2022,"$92,368",0%,United States,Columbus,NaN
1,E02003,Robert Patel,Analyst,Sales,Corporate,Male,Asian,58.0,10/23/2013,"$45,703",0%,United States,Chicago,NaN
2,E02004,Cameron Lo,Network Administrator,IT,Research & Development,Male,Asian,34.0,3/24/2019,"$83,576",0%,China,Shanghai,NaN
3,E02005,Harper Castillo,IT Systems Architect,IT,Corporate,Female,Latino,39.0,4/7/2018,"$98,062",0%,United States,Seattle,NaN
4,E02006,Harper Dominguez,Director,Engineering,Corporate,Female,Latino,42.0,6/18/2005,"$175,391",24%,United States,Austin,NaN


In [4]:
df.tail()

,Employee ID,Full Name,Job Title,Department,Business Unit,Gender,Ethnicity,Age,Hire Date,Annual Salary,Bonus %,Country,City,Exit Date
1257,E02250,Mila Han,Manager,Sales,Manufacturing,Female,Asian,54.0,11/14/2009,"$128,791",6%,United States,Miami,NaN
1258,E02251,Genesis Herrera,Manager,IT,Research & Development,Female,Latino,34.0,10/3/2015,"$126,898",10%,Brazil,Manaus,NaN
1259,E02252,Olivia Vazquez,Network Engineer,IT,Specialty Products,Female,Latino,53.0,4/13/2020,"$93,053",0%,Brazil,Sao Paulo,NaN
1260,E02253,Leilani Ng,Systems Analyst,IT,Corporate,Female,Asian,48.0,9/19/2011,"$50,513",0%,United States,Seattle,10/30/2019
1261,E02254,Olivia Mendoza,Sr. Account Representative,Sales,Corporate,Female,Latino,43.0,5/7/2017,"$86,533",0%,United States,Columbus,NaN


In [5]:
date_cols = ['Hire Date', 'Exit Date']

for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

In [7]:
df['Bonus %'] = pd.to_numeric(df['Bonus %'].str.rstrip('%'))
df.rename(columns={'Bonus %' : 'Bonus'}, inplace=True)

In [10]:
df['Annual Salary'] = pd.to_numeric(df['Annual Salary'].str.replace('[^\d]', '', regex=True), errors='coerce')

In [13]:
df['Age'] = df['Age'].astype('Int64')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1262 entries, 0 to 1261
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Employee ID    1243 non-null   object        
 1   Full Name      1222 non-null   object        
 2   Job Title      1204 non-null   object        
 3   Department     1215 non-null   object        
 4   Business Unit  1180 non-null   object        
 5   Gender         1213 non-null   object        
 6   Ethnicity      1220 non-null   object        
 7   Age            1256 non-null   Int64         
 8   Hire Date      1227 non-null   datetime64[ns]
 9   Annual Salary  1189 non-null   float64       
 10  Bonus          1214 non-null   float64       
 11  Country        1156 non-null   object        
 12  City           1207 non-null   object        
 13  Exit Date      125 non-null    datetime64[ns]
dtypes: Int64(1), datetime64[ns](2), float64(2), object(9)
memory usage: 139.

In [ ]:
duplicates = df.duplicated(subset='Employee ID', keep = False)

In [24]:
sorted_dups = duplicates.sort_values('Employee ID')

In [27]:
df['Employee ID'].isnull().sum()

19

In [28]:
sorted_dups

# we can conclude that same values are repeated for all the columns so we can drop them
# and here not considering the null values, they are only 19 so i wanna get rid of them also

,Employee ID,Full Name,Job Title,Department,Business Unit,Gender,Ethnicity,Age,Hire Date,Annual Salary,Bonus,Country,City,Exit Date
0,E02002,Kai Le,Controls Engineer,Engineering,Manufacturing,Male,Asian,47,2022-02-05,92368.0,0.0,United States,Columbus,NaT
1173,E02002,Kai Le,Controls Engineer,Engineering,Manufacturing,Male,Asian,47,2022-02-05,92368.0,0.0,NaN,NaN,NaT
1000,E02002,Kai Le,NaN,Engineering,Manufacturing,Male,Asian,47,2022-02-05,92368.0,0.0,NaN,Columbus,NaT
1,E02003,Robert Patel,Analyst,Sales,Corporate,Male,Asian,58,2013-10-23,45703.0,0.0,United States,Chicago,NaT
1174,E02003,Robert Patel,Analyst,Sales,Corporate,Male,Asian,58,2013-10-23,45703.0,0.0,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,NaN,NaN,Director,Human Resources,Specialty Products,Female,Caucasian,53,NaT,NaN,NaN,United States,Miami,NaT
983,NaN,Logan Rahman,Manager,NaN,Specialty Products,Male,Asian,39,NaT,NaN,NaN,United States,Austin,NaT
984,NaN,Henry Rogers,Sr. Account Representative,Sales,NaN,NaN,Caucasian,25,NaT,72693.0,NaN,NaN,Seattle,NaT
1020,NaN,Adrian James,Enterprise Architect,IT,Research & Development,Male,Caucasian,52,1994-08-15,73779.0,0.0,United States,Chicago,NaT


In [30]:
df.drop_duplicates(subset='Employee ID', keep="first", inplace=True)

In [31]:
df['Employee ID'].isnull().sum()

1

In [34]:
df.dropna(subset=['Employee ID'], inplace=True)

In [42]:
sorted_df = df.sort_values('Employee ID').reset_index(drop=True)

In [43]:
sorted_df

,Employee ID,Full Name,Job Title,Department,Business Unit,Gender,Ethnicity,Age,Hire Date,Annual Salary,Bonus,Country,City,Exit Date
0,E02002,Kai Le,Controls Engineer,Engineering,Manufacturing,Male,Asian,47,2022-02-05,92368.0,0.0,United States,Columbus,NaT
1,E02003,Robert Patel,Analyst,Sales,Corporate,Male,Asian,58,2013-10-23,45703.0,0.0,United States,Chicago,NaT
2,E02004,Cameron Lo,Network Administrator,IT,Research & Development,Male,Asian,34,2019-03-24,83576.0,0.0,China,Shanghai,NaT
3,E02005,Harper Castillo,IT Systems Architect,IT,Corporate,Female,Latino,39,2018-04-07,98062.0,0.0,United States,Seattle,NaT
4,E02006,Harper Dominguez,Director,Engineering,Corporate,Female,Latino,42,2005-06-18,175391.0,24.0,United States,Austin,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980,E02997,Melody Delgado,Network Architect,IT,Specialty Products,Female,Latino,48,2012-11-28,NaN,0.0,NaN,Rio de Janeiro,NaT
981,E02998,Eliza Bui,Analyst,Finance,Corporate,Female,NaN,63,2010-06-13,NaN,0.0,NaN,NaN,NaT
982,E02999,Layla Wu,NaN,Sales,Research & Development,Female,Asian,31,2018-08-27,NaN,0.0,United States,NaN,NaT
983,E03000,Thomas Lam,NaN,Engineering,Manufacturing,Male,Asian,52,2022-11-18,NaN,NaN,United States,NaN,NaT


In [77]:
sorted_df.columns = sorted_df.columns.str.replace(' ', '_')

In [78]:
sorted_df.columns

Index(['Employee_ID', 'Full_Name', 'Job_Title', 'Department', 'Business_Unit',
       'Gender', 'Ethnicity', 'Age', 'Hire_Date', 'Annual_Salary', 'Bonus',
       'Country', 'City', 'Exit_Date'],
      dtype='object')

In [88]:
sorted_df.head()
sorted_df['Employee_ID'] = sorted_df['Employee_ID'].astype(str)



In [89]:
sorted_df.head()

,Employee_ID,Full_Name,Job_Title,Department,Business_Unit,Gender,Ethnicity,Age,Hire_Date,Annual_Salary,Bonus,Country,City,Exit_Date
0,E02002,Kai Le,Controls Engineer,Engineering,Manufacturing,Male,Asian,47,2022-02-05,92368.0,0.0,United States,Columbus,NaT
1,E02003,Robert Patel,Analyst,Sales,Corporate,Male,Asian,58,2013-10-23,45703.0,0.0,United States,Chicago,NaT
2,E02004,Cameron Lo,Network Administrator,IT,Research & Development,Male,Asian,34,2019-03-24,83576.0,0.0,China,Shanghai,NaT
3,E02005,Harper Castillo,IT Systems Architect,IT,Corporate,Female,Latino,39,2018-04-07,98062.0,0.0,United States,Seattle,NaT
4,E02006,Harper Dominguez,Director,Engineering,Corporate,Female,Latino,42,2005-06-18,175391.0,24.0,United States,Austin,NaT


In [90]:
sorted_df['Employee_ID'].dtype

dtype('O')

In [54]:
hostname = "localhost"
username = "root"
password = "tony7092"
database = "employee_management"

In [55]:
my_db = mysql.connector.connect(
    host = hostname,
    user = username,
    passwd = password,
    database = database
)

In [73]:
my_db.is_connected()

False

In [85]:
if my_db.is_connected():
    
    my_engine = create_engine(f'mysql+mysqlconnector://{username}:{password}@{hostname}/{database}')
    table_name = 'Employee'
    sorted_df.to_sql(name=table_name, con=my_engine, if_exists='replace', index=False)
    print("Successfully converted dataframe to mysql table")
    my_db.close()
else:
    print("Connection to mysql failed.")

Successfully converted dataframe to mysql table


C:\Users\TONY BHASKAR\AppData\Local\Temp\ipykernel_8292\3894946086.py:5: UserWarning: The provided table name 'Employee' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  sorted_df.to_sql(name=table_name, con=my_engine, if_exists='replace', index=False)


In [84]:
my_db.reconnect()